In [34]:
from typing import Optional, Tuple

import cv2
import numpy as np

import sys
np.set_printoptions(threshold=sys.maxsize)

from pyk4a import ImageFormat

def colorize(
    image: np.ndarray,
    clipping_range: Tuple[Optional[int], Optional[int]] = (None, None),
    colormap: int = cv2.COLORMAP_HSV,
) -> np.ndarray:

    resize_ratio = 2

    if clipping_range[0] or clipping_range[1]:
        img = image.clip(clipping_range[0], clipping_range[1])
    else:
        img = image.copy()
    img1 = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    img2 = cv2.applyColorMap(img1, colormap)
    
    img3 = cv2.resize(img2, (img2.shape[1]//resize_ratio, img2.shape[0]//resize_ratio))
    return img3


In [9]:
 """
        np array of floats
        (n_bodies, n_joints, n_data) == body_skeleton.shape
        Position_image are in color image coordinates.
        if valid != 1: position does not fit in image.
        data for a joint follows this order(
            position_x,
            position_y,
            position_z,
            orientation_w,
            orientation_x,
            orientation_y,
            orientation_z,
            confidence_level,
            position_image_0,
            position_image_1,
            valid,
        )
"""
    
"""
Index           Joint name          Parent joint
0               PELVIS         
1	            SPINE_NAVAL	        PELVIS
2	            SPINE_CHEST	        SPINE_NAVAL
3	            NECK	            SPINE_CHEST
4	            CLAVICLE_LEFT	    SPINE_CHEST
5	            SHOULDER_LEFT   	CLAVICLE_LEFT
6	            ELBOW_LEFT      	SHOULDER_LEFT
7	            WRIST_LEFT      	ELBOW_LEFT
8	            HAND_LEFT       	WRIST_LEFT
9	            HANDTIP_LEFT    	HAND_LEFT
10	            THUMB_LEFT      	WRIST_LEFT
11	            CLAVICLE_RIGHT     	SPINE_CHEST
12	            SHOULDER_RIGHT  	CLAVICLE_RIGHT
13	            ELBOW_RIGHT     	SHOULDER_RIGHT
14	            WRIST_RIGHT     	ELBOW_RIGHT
15	            HAND_RIGHT      	WRIST_RIGHT
16	            HANDTIP_RIGHT   	HAND_RIGHT
17	            THUMB_RIGHT	        WRIST_RIGHT
18	            HIP_LEFT        	PELVIS
19	            KNEE_LEFT       	HIP_LEFT
20	            ANKLE_LEFT      	KNEE_LEFT
21	            FOOT_LEFT       	ANKLE_LEFT
22	            HIP_RIGHT       	PELVIS
23	            KNEE_RIGHT      	HIP_RIGHT
24	            ANKLE_RIGHT     	KNEE_RIGHT
25	            FOOT_RIGHT      	ANKLE_RIGHT
26	            HEAD            	NECK
27	            NOSE            	HEAD
28	            EYE_LEFT        	HEAD
29	            EAR_LEFT        	HEAD
30	            EYE_RIGHT       	HEAD
31	            EAR_RIGHT       	HEAD
"""


import cv2

import pyk4a
# from helpers import colorize
from pyk4a import ColorResolution, Config, PyK4A


k4a = PyK4A(
    Config(
        color_resolution=ColorResolution.RES_720P,
        depth_mode=pyk4a.DepthMode.NFOV_UNBINNED,
        camera_fps=pyk4a.FPS.FPS_30,
    )
)
k4a.start()

def process_webcam(args, width=500, height=300):
    fps = args["WEBCAM_FPS"]
    win_size = int(args["WIN_SIZE"] * fps)
    strides = int(win_size * args["STRIDE_RATIO"])
    frames = []
    
    count = 0
    prev = 0
    state = 1

    while True:
        capture = k4a.get_capture()
        body_skeleton = capture.body_skeleton
        
#         width = capture.get_width_pixels()
#         height = capture.get_height_pixels()
        if body_skeleton is None:
            continue

#         frame = colorize(capture.transformed_depth, (None, 5000))
#         blank_frame = np.zeros(shape=[width, height, 3], dtype=np.uint8)
        resize_ratio = 1
        frame = capture.color
        frame = cv2.resize(frame, (frame.shape[1]//resize_ratio, frame.shape[0]//resize_ratio))
#         frame = capture.transformed_depth

        for body_index in range(body_skeleton.shape[0]):
            skeleton = body_skeleton[body_index, :, :]
#             print(skeleton)
#             print()
            for joint_index in range(skeleton.shape[0]):
                valid = int(skeleton[joint_index, -1])
                if valid != 1:
                    continue
                x, y = skeleton[joint_index, (-3, -2)].astype(int)
                x, y = x//resize_ratio, y//resize_ratio
#                     print(x, y)
                cv2.circle(frame, (x, y), 3, (50, 50, 50), thickness=-1, lineType=cv2.FILLED)
                cv2.putText(frame, str(joint_index), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2, cv2.LINE_AA)
        
#         # squat rule
        
#         skeleton2 = body_skeleton[0, :, :]
#         pelvic_x, pelvic_y = skeleton[0, (-3, -2)].astype(int)
#         knee_x, knee_y = skeleton[19, (-3, -2)].astype(int)
        
#         if state == 1 and pelvic_y < knee_y:
#             state = 2
#         if state == 2 and pelvic_y > knee_y:
#             state = 1
#             count += 1
        
# #         print("pelvic_y", pelvic_y)
# #         print("knee_y", knee_y)
        
#         if(prev != count):
#             print("count: ", count)
            
            
#         prev = count
        
        cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            break

    k4a.stop()
    cv2.destroyAllWindows()

K4AException: 

In [10]:
##@title 
args = {
    "WEBCAM_FPS" : 30, # FPS while recording video from webcam.
    "RECORDING_TIME_IN_SECONDS" : 1.0, # Time in seconds to record video on webcam.
    "THRESHOLD" : 0.2, # Threshold to consider periodicity in entire video.
    "WITHIN_PERIOD_THRESHOLD" : 0.2, # Threshold to consider periodicity for individual frames in video.
    "CONSTANT_SPEED" : False,
    "MEDIAN_FILTER": True, # Use median filtering in time to ignore noisy frames.
    "FULLY_PERIODIC": False,
    "PLOT_SCORE" : True,
    "VIZ_FPS" : 30,
    "WIN_SIZE" : 1,
    "STRIDE_RATIO": 0.5, # Window stride ratio respect to win-size for webcam mode
    "STRIDES" : [1],
    "VIZ_REPS" : True,
    "ROT": None, # Rotate videos. (none, cw, ccw, 180)
    "PATH" : "squat.mp4",
    "OUT" : "./export",
    "WEIGHTS_FOLDER" : "./weights",
    "CAMERA_NUMBER": 0
}


In [11]:
# if __name__ == '__main__':
   
#     process_webcam(args)

In [12]:
# import cv2
# import numpy as np

# import pyk4a
# from pyk4a import Config, PyK4A


# def main():
#     k4a = PyK4A(
#         Config(
#             color_resolution=pyk4a.ColorResolution.OFF,
#             depth_mode=pyk4a.DepthMode.NFOV_UNBINNED,
#             synchronized_images_only=False,
#         )
#     )
#     k4a.start()

#     # getters and setters directly get and set on device
#     k4a.whitebalance = 4500
#     assert k4a.whitebalance == 4500
#     k4a.whitebalance = 4510
#     assert k4a.whitebalance == 4510

#     while True:
#         capture = k4a.get_capture()
#         if np.any(capture.depth):
#             cv2.imshow("k4a", colorize(capture.depth, (None, 5000), cv2.COLORMAP_HSV))
#             key = cv2.waitKey(10)
#             if key != -1:
#                 cv2.destroyAllWindows()
#                 break
#     k4a.stop()


# if __name__ == "__main__":
#     main()

In [42]:
import cv2

import pyk4a
from pyk4a import Config, PyK4A


depth_threshold = 10000

def main():
    k4a = PyK4A(Config(color_resolution=pyk4a.ColorResolution.RES_720P, depth_mode=pyk4a.DepthMode.NFOV_UNBINNED,))
    k4a.start()

    while True:
        capture = k4a.get_capture()
#         if capture.depth is not None:
#             cv2.imshow("Depth1", colorize(capture.depth, (None, 500)))
#             cv2.imshow("Depth2", colorize(capture.depth, (None, 1000)))
#             cv2.imshow("Depth3", colorize(capture.depth, (None, 5000)))
#             cv2.imshow("Depth4", colorize(capture.depth, (None, 10000)))
#         if capture.color is not None:
#             cv2.imshow("Color", capture.color)
        if capture.transformed_depth is not None:
#             cv2.imshow("Transformed Depth1", colorize(capture.transformed_depth, (None, 500)))
            cv2.imshow("Transformed Depth2", colorize(capture.transformed_depth, (None, 3000)))
#             cv2.imshow("Transformed Depth3", colorize(capture.transformed_depth, (None, 5000)))
#             cv2.imshow("Transformed Depth4", colorize(capture.transformed_depth, (None, 10000)))
#         if capture.transformed_color is not None:
#             cv2.imshow("Transformed Color", capture.transformed_color)
        key = cv2.waitKey(10)
        if key != -1:
            cv2.destroyAllWindows()
            break

    k4a.stop()


if __name__ == "__main__":
    main()